In [ ]:
import h5py
path = "/Volumes/HEIU/data/bulkice_train.hdf5"
print(path)
f = h5py.File(path, "r")


In [ ]:
from dataset import AFMDataset
path = "/Volumes/HEIU/data/bulkice_train.hdf5"

dts = AFMDataset(path)
fn, afm, label_type, label_pos = dts[0]

In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
plt.imshow(make_grid(afm.transpose(1,0)).permute(1,2,0))

In [ ]:
f['T180_13015'].afm

In [ ]:
from dataset.dataset import z_sampler
z_sampler(10, 4, True)

In [ ]:
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt
import time
from dataset import transform as tf

pics = torch.ones(1, 8, 100, 100) * 0.9
pics[..., 50:, 0:50] = 0.1
pics[..., :50, 50:] = 0.1

transform = nn.Sequential(
    tf.PixelShift(),
    tf.Cutout(),
    tf.ColorJitter(),
    tf.Noisy(),
    tf.Blur(),
)
t = time.time()
for i in range(1):
    pics = transform(pics)

print(time.time() - t)


T = torchvision.utils.make_grid(pics.transpose(1,0),pad_value = 0.5)
plt.imshow(T.permute(1,2,0))


In [ ]:
import torch
torch.arange(10)

In [ ]:
a = torch.ones(100, 100)
a[10:20, 10:20]

In [ ]:
import torch
a = torch.rand(100, 100, 3)
b = torch.rand(100, 100) > 0.5
a[b]

In [ ]:
import torch
a = torch.ones((50, 50, 5))
c = torch.rand((50, 50, 5, 3))
mask = a.nonzero()
print(mask.T.shape)
a[tuple(mask.T)]
(c[tuple(mask.T)] + mask) / (50, 50, 5)

In [ ]:
import torch
from utils import poscar
box_cls = torch.randint(0, 3, (3, 50, 50), dtype=torch.long)
box_off = torch.rand((3, 50, 50, 3))
print(poscar.boxncls2pos_torch(box_cls, box_off))
print(poscar.boxncls2pos_np(box_cls.numpy(), box_off.numpy()))

In [ ]:
import torch
from utils.metrics import Analyser
from utils import poscar
from dataset import AFMDataset
path = "/Volumes/HEIU/data/bulkice_train.hdf5"

dts = AFMDataset(path)
fn, afm, label_type, label_pos = dts[0]
pred_types, pred_pos = poscar.targ2pred(label_type, label_pos)
a = torch.jit.script(Analyser())
pred_types = pred_types.unsqueeze(0).repeat((10, 1, 1, 1, 1))
pred_pos  = pred_pos.unsqueeze(0).repeat((10, 1, 1, 1, 1))
label_type = label_type.unsqueeze(0).repeat((10, 1, 1, 1))
label_pos = label_pos.unsqueeze(0).repeat((10, 1, 1, 1, 1))

In [ ]:
import time
t = time.time()
for i in range(1000):
    a(pred_types, pred_pos, label_type, label_pos)
print(time.time() - t)

In [ ]:
a = torch.rand((100, 100, 3))
mask = torch.nonzero(a, as_tuple=True)
a[mask[0], mask[1], mask[2]]

In [ ]:
import time
import torch
from utils.metrics import ConfusionMatrixCounter
cm = torch.randint(0, 100, (1, 2, 1, 3)).numpy()
t = time.time()
ConfusionMatrixCounter._count(cm)

In [ ]:
import torch
from model import unet_onehot
m = unet_onehot()
img = torch.rand((1, 1, 10, 100, 100))
m(img)

In [ ]:
import torch
import numpy as np
from utils.metrics import Analyser
from utils import poscar
from dataset import AFMDataset
path = "/Volumes/HEIU/data/bulkice_train.hdf5"

dts = AFMDataset(path)
fn, afm, label_type, label_pos = dts[0]
print(fn)
typ, final, _ = poscar.boxncls2pos_torch(label_type, label_pos)
typ = typ.numpy()
final = final.numpy()
print(len(final))
pos2 = np.concatenate([final[typ == i] for i in range(1,3)])
poscar.save("test.poscar", [3.0, 25.0, 25.0], ['O', 'H'], [48, 96], pos2)

In [ ]:
import numpy as np
a = ["1", "2", "3"]
np.array(a, dtype=np.float32)

In [ ]:
int("9246.0")

In [134]:
import numpy as np
from numba import jit




# @jit(nopython=True)
def getWaterRotate(tag: np.ndarray, inv_ref: np.ndarray) -> np.ndarray:
    O, H1, H2 = tag
    H1, H2 = H1 - O, H2 - O
    H1 = H1
    H2 = H2
    H3 = np.cross(H1, H2)
    if H3[2] < 0:
        tag[0] = H2
        tag[1] = H1
        tag[2] = -H3
    else:
        tag[0] = H1
        tag[1] = H2
        tag[2] = H3
    return inv_ref @ tag

ang = 104.52 / 180 * np.pi
# v = np.array([0, 0, 1]) * 0.9572
# u = np.array([np.sin(ang), 0, np.cos(ang)]) * 0.9572
# r = np.cross(v, u)
# print(v, u,r)
initpos = np.array([
        [ 0.         , 0.         , 0.        ],
        [ 0.         , 0.         , 0.9572    ],
        [ 0.9266272  , 0.         ,-0.23998721],
    ])

initdir = np.array([
        [ 0.         , 0.         , 0.9572    ],
        [ 0.9266272  , 0.         ,-0.23998721],
        [ 0.         , 0.88696756 , 0.        ]
    ])



pos = np.asarray([[
        [6.20233009,  6.80328286,  0.71540000], 
        [6.72382009,  6.43174286,  0.00390000], 
        [5.46304009,  6.20268286,  0.81010000]
        ],
        [
        [14.10287009, 18.97821286,  0.47590000], 
        [14.35087009, 19.79591286,  0.04450000], 
        [13.24817009, 19.16231286,  0.86570000]
        ]])

def getRotate(final: np.ndarray, init: np.ndarray) -> np.ndarray:
    """
    Rotate init to final, init @ R = final

    Args:
        final (np.ndarray): N 3 3
        init (np.ndarray): 3 3

    Returns:
        np.ndarray: N 3 3
    """
    init = np.linalg.inv(init)
    final = final - final[:, (0,), :]
    cross = np.cross(final[:,1], final[:, 2])
    
    final = np.stack([final[:,1], final[:, 2], cross], axis=1)
    R = np.einsum("ij, Njk -> Nik", init, final)
    # print(np.linalg.det(R))
    return R
    
R = getRotate(pos, initdir)

R1 = R[0]
print(initpos @ R1)
# print(np.einsum("Nij, kj -> Nki", R, initpos))

import time
t = time.time()
R2 = getWaterRotate(pos[0].copy(), np.linalg.inv(initdir))
print(time.time() - t)

print(initpos @ R2[(0,2,1),:])
print(initpos @ R2)
# print(pos[0] - pos[0,(0,), :])

[[ 0.       0.       0.     ]
 [ 0.52149 -0.37154 -0.7115 ]
 [-0.73929 -0.6006   0.0947 ]]
0.0001270771026611328
[[ 0.          0.          0.        ]
 [ 0.49913464 -0.51435974  0.63443281]
 [ 0.21099473 -0.39316198 -0.8468207 ]]
[[ 0.       0.       0.     ]
 [-0.73929 -0.6006   0.0947 ]
 [ 0.52149 -0.37154 -0.7115 ]]


In [131]:
print(np.cross(R1[0], R1[1]) - R1[2])
print(np.cross(R2[0], R2[1]) - R2[2])
print(R1)
print(R2)
print(R2[(0,-2,-1),:])

[-4.88610450e-05 -3.39947613e-05  1.08801175e-05]
[ 4.60969633e-05 -8.14802859e-06 -4.28723244e-05]
[[-0.65672916 -0.74868485 -0.09031228]
 [-0.52145282  0.5373587  -0.66280068]
 [ 0.54480777 -0.38815295 -0.74331383]]
[[ 0.36275293 -0.56346413 -0.74221544]
 [ 0.52145282 -0.5373587   0.66280068]
 [-0.77234643 -0.62745508  0.09893439]]
[[ 0.36275293 -0.56346413 -0.74221544]
 [ 0.52145282 -0.5373587   0.66280068]
 [-0.77234643 -0.62745508  0.09893439]]


In [ ]:
import torch
from model import VAEunet
import utils
net = VAEunet(channel_mult = (1, 2, 3, 4))
temp = torch.rand((1, 1))
print("\n".join(utils.model.model_structure(net)))
image = torch.rand((1, 10, 4,25, 25))
mu, nu, x = net(image, temp)

In [ ]:
from dataset import AFMGenDataset
dts = AFMGenDataset("/Volumes/HEIU/data/ice_8_4A/ice_8_4A_train.hdf5")
name, box4a, box8a = dts[0]

In [ ]:
print(len(box4a[...,0].nonzero()))
print(len(box8a[...,0].nonzero()))

In [62]:
import torch
a = torch.zeros(10)
a[(0,1,2),] = 1
print(a)

tensor([1., 1., 1., 0., 0., 0., 0., 0., 0., 0.])


In [ ]:
def argmatch(pred: Tensor, targ: Tensor, cutoff: float) -> tuple[Tensor]:
    dis = torch.cdist(targ, pred)
    dis = (dis < cutoff).nonzero()
    dis = dis[:, (1, 0)]
    unique, idx, counts = torch.unique(dis[...,1], sorted=True, return_inverse=True, return_counts=True)
    ind_sorted = torch.argsort(idx, stable=True)
    cum_sum = counts.cumsum(0)
    cum_sum = torch.cat((torch.tensor([0]), cum_sum[:-1]))
    first_indicies = ind_sorted[cum_sum]
    dis = dis[first_indicies]
    return dis

import torch
b = torch.rand(1000, 3)
c = torch.rand(100, 3)
t = time.time()
for i in range(1000):
    reg = argmatch(b, c, 0.1)
print(time.time() - t)
reg = argmatch(b, c, 0.1).T
print(reg)
print(reg[0].unique().shape, reg[0].shape)


In [2]:

import torch
a = torch.tensor([1.0])
b = a.to("mps")

tensor([1.])

In [4]:
a = [1, 2, 3]
b = [1]
a + b

[1, 2, 3, 1]

In [22]:
import numpy as np
a = np.zeros((10,10))
b = []
b.extend(a)

import torch
# 0.0, 0.5, 1.0; 1.0, 2.0, 3.0
# 0.0, 0.5, 1.0; 0.0, 1.0, 2.0
a = torch.linspace(0, 1, 3)[None, None, :].repeat(1, 3, 1)
a[:, 1] += 1
b = torch.linspace(1, 3, 3)[None, None, :].repeat(1, 3, 1)
c.arccos_().div_(torch.pi).mul_(180)
print(c)

tensor([[17.0239,  6.9825, 17.0239]])


In [ ]:
import multiprocessing as mp
import time

def f():
    print("Zzz")
    time.sleep(5)

p = mp.Pool(6)

_ = [p.apply_async(f) for i in range(12)]

In [50]:
a = np.zeros((3,3))
a[0] = 1
b = np.random.rand(3,3)
print(a)
print(b)
print(np.matmul(a,b))

[[1. 1. 1.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0.53401544 0.3812444  0.08377378]
 [0.99078535 0.02127975 0.12840506]
 [0.07252139 0.3136849  0.3243425 ]]
[[1.59732218 0.71620905 0.53652133]
 [0.         0.         0.        ]
 [0.         0.         0.        ]]


In [4]:
from dataset import AFMGenDataset
from utils import functional
import torch
a  = AFMGenDataset("/Users/supercgor/Documents/data/ice_8_4A/ice_8_4A_small_train.hdf5")
filename ,inp, targ, emb = a[0]
conf, pos, rot = functional.box2orgvec(targ, 0.5, 1.0, (25.0,25.0, 4.0), sort = False, nms = False)



In [4]:
import torch
import numpy as np
pos = torch.tensor([[23.5279, 19.8613,  0.7911]])
rot = torch.tensor([[[ 0.1403,  0.2997, -0.9437],
        [-0.5516,  0.8152,  0.1769],
        [ 0.8223,  0.4957,  0.2797]]])
from utils.functional import getWaterRotate, makewater
print(pos.shape)
print(makewater(pos,rot))
initdir = np.array([
        [ 0.         , 0.         , 0.9572    ],
        [ 0.9266272  , 0.         ,-0.23998721],
        [ 0.         , 0.88696756 , 0.        ]
        ])
invref = np.linalg.inv(initdir)
pos = np.array([[23.52787009, 19.86131286,  4.7911    ],
 [24.31487009, 20.33571286,  5.0589    ],
 [23.46057009, 20.02011286,  3.8495    ]])
print(pos-pos[0])
getWaterRotate(pos, invref)



torch.Size([1, 3])
[[[ 0.          0.          0.        ]
  [ 0.78710558  0.47448404  0.26772885]
  [-0.84392866  0.4283606   0.01057266]]]
[[[23.5279007  19.86129951  0.79110003]
  [24.31500627 20.33578356  1.05882888]
  [22.68397204 20.28966012  0.80167268]]]
[[ 0.      0.      0.    ]
 [ 0.787   0.4744  0.2678]
 [-0.0673  0.1588 -0.9416]]


array([[ 0.14030995,  0.29973283, -0.94369962],
       [-0.5515666 ,  0.81515525,  0.17689792],
       [ 0.82218972,  0.4956122 ,  0.27977434]])

In [6]:
from dataset import AFMGenDataset
from utils import functional
import torch
a  = AFMGenDataset("/Users/supercgor/Documents/data/ice_8_4A/ice_8_4A_small_train.hdf5")
filename ,inp, targ, emb = a[0]

targ2 = functional.box2box(targ)

torch.allclose(targ, targ2, rtol = 1e-4)

True

In [2]:
a = torch.ones_like(inp, (1,2,3))

TypeError: ones_like() takes 1 positional argument but 2 were given

In [10]:
a  = [1, 2, 3]
b = list(map(lambda x: x + 1, a))
print(a, b)


[1, 2, 3] [2, 3, 4]


In [1]:
import torch
from model import VAELatEmb
a = VAELatEmb()
b = torch.randn((2, 10, 4, 25, 25))
c = torch.randn((2, 1))
out, mu, nu = a(b, c)
out.shape

torch.Size([2, 10, 8, 25, 25])